In [2]:
import os
ON_KAGGLE = os.path.exists('/kaggle/input')
ENABLE_SUBMISSION = False
os.environ["ON_KAGGLE"] = str(ON_KAGGLE)

In [3]:
# Install required packages if on Kaggle
if ON_KAGGLE:
    !pip install -U /kaggle/input/accelerate-wheelwhl/accelerate-0.29.1-py3-none-any.whl -qq
    !pip install -U /kaggle/input/bitsandbytes-0-42-0-py3-none-any-whl/bitsandbytes-0.42.0-py3-none-any.whl -qq
    !pip install -U /kaggle/input/hydracore120py3/hydra_core-1.2.0-py3-none-any.whl -qq
    !pip install -U /kaggle/input/omegaconf222py3/omegaconf-2.2.2-py3-none-any.whl -qq
    !pip install -U /kaggle/input/parglare-0-18-0/parglare-0.18.0-py3-none-any.whl -qq
    !pip install -U /kaggle/input/rank-bm25-whl/rank_bm25-0.2.2-py3-none-any.whl -qq
    !pip install -U /kaggle/input/peft-0-11-1/peft-0.11.1-py3-none-any.whl -qq
    !pip install -U /kaggle/input/trl-0-9-4/trl-0.9.4-py3-none-any.whl -qq
    !pip install -U /kaggle/input/amino-gaz-whl/aimo_gaz-0.0.1-py3-none-any.whl -qq --no-deps --force-reinstall
else:
    !pip install -r requirements.txt
    !pip install --upgrade build
    !python -m build
    !pip install dist/aimo_gaz-0.0.1-py3-none-any.whl --force-reinstall

In [7]:
from aimo_gaz.models.model import Model

2024-06-17 03:16:27.387135: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-17 03:16:27.387284: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-17 03:16:27.609610: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
# Model from Hugging Face hub
# Test model
import os
import numpy as np
import json
model_name = "Salesforce/codet5-small"
is_seq2seq = "llama-2" not in model_name.lower()
token = None
model = Model(model_name, token=token, use_lora=False, is_seq2seq=is_seq2seq)
main_prompt = "Do simple math problems (Answer only the number and use '[END]' to finish the response):\nQuestion: 2 + 2\nAnswer: 4\n[END]"
with model:
    for response in model.generate(
            [
                f"{main_prompt}\nQuestion: 4 + 5\nAnswer:",
                f"{main_prompt}\nQuestion: 2 + 2\nAnswer:",
                f"{main_prompt}\nQuestion: 3 + 3 * 3\nAnswer:",
            ],
            max_new_tokens=10,
            temperature=0.1, # Nucleus sampling
            do_sample=True, # Nucleus sampling
            top_k=5, # Nucleus sampling
            # num_beams=5, # Beam search
            num_return_sequences=5,
            stop_tokens=["[END]"],#, model._tokenizer.eos_token],
            skip_special_tokens=True,
            padding=True,
            #truncation=True,
            compute_probabilities=True,
            return_full_text=False):

        print("-" * 50)
        print(f"Prompt: \n{response.input_text}")
        for idx, result in enumerate(response.generated_text):
            probability = np.exp(-response.neg_log_likelihood[idx])
            print(f"Result [{idx + 1}] ({probability*100}% chance): {result}")
        print("-" * 50)

--------------------------------------------------
Prompt: 
Do simple math problems (Answer only the number and use '[END]' to finish the response):
Question: 2 + 2
Answer: 4
[END]
Question: 4 + 5
Answer:
Result [1] (12.239348058612599% chance): +
Result [2] (12.239348058612599% chance): +
Result [3] (12.239348058612599% chance): +
Result [4] (12.239348058612599% chance): +
Result [5] (12.239348058612599% chance): +
--------------------------------------------------
--------------------------------------------------
Prompt: 
Do simple math problems (Answer only the number and use '[END]' to finish the response):
Question: 2 + 2
Answer: 4
[END]
Question: 2 + 2
Answer:
Result [1] (11.436164139065806% chance): 4
Result [2] (11.436164139065806% chance): 4
Result [3] (11.436164139065806% chance): 4
Result [4] (11.059441369837298% chance): + 2
Result [5] (11.293735083530933% chance): 4
[END]
--------------------------------------------------
--------------------------------------------------

In [12]:
from aimo_gaz.solver.abs_solver import Solver
solver = Solver()

TypeError: Can't instantiate abstract class Solver with abstract method solve

In [14]:
# Set up the evaluation API
if ENABLE_SUBMISSION and ON_KAGGLE:
    import aimo

    env = aimo.make_env()
    iter_test = env.iter_test()

    # Iterate through the test set and use the model make predictions
    for test, sample_submission in iter_test:
        sample_submission['answer'] = solver.solve(text['problem'])
        env.predict(sample_submission)
        print(test)
        print(sample_submission, '\n')